<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai
%pip -q install google-adk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# E necessário inserir a chave na variável do seu ambiente com sua GOOGLE_API_KEY
google_api_key = os.environ.get('GOOGLE_API_KEY')
if google_api_key:
    print("Chave da API do Google encontrada nas variáveis de ambiente do seu PC.\n")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente e tentaremos pelo GOOGLE COLAB.\n")
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Chave da API do Google encontrada nas variáveis de ambiente do seu PC.



In [3]:
from google import genai
client = genai.Client()
modelo = "gemini-2.0-flash"

In [4]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [5]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [6]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [13]:
##########################################
# --- Agente 1: BUSCADOR --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pesquisa de móveis ou produtos e deve usar a ferramenta de
      busca do Google (google_search), procure pelo móvel e que tenha preço e sejam o mais barato,
      faça uma planilha de preços no final.
      Que tenha garantia e boa referência de reputação e avaliação a partir da data de hoje.
      Somente guarde resultados que possuem preços e links dos produtos.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  moveis = call_agent(buscador, entrada_do_agente_buscador)
  return moveis

In [14]:
################################################
# --- Agente 2: ORÇAMENTO --- #
################################################
def agente_cotador(topico, moveis_buscados):
    planejador = Agent(
        name="agente_cotacao",
        model="gemini-2.0-flash",
        description="Agente cotação de preços",
        tools=[google_search],
        instruction="""
        Você é um filtro de pesquisa, especialista em encontrar o móvel ou produtos com melhor custo
        benefícioa partir da data de hoje. Você deve usar a feramenta de busca do Google (google_search) e dar
        prioridade preço mais baixo.
        Escolha de 1 até 5 móveis ou produtos que tenham preços e com formas de pagamentos se houver e seus links.
        """
    )
    entrada_do_agente_cotador = f"Tópico:{topico}\nLançamentos buscados: {moveis_buscados}"

    moveis_do_post = call_agent(planejador, entrada_do_agente_cotador)
    return moveis_do_post

In [9]:
######################################
# --- Agente 3: AVALIADOR --- #
######################################
def agente_avaliador(topico, avaliacao_de_post):
    redator = Agent(
        name="agente_avaliador",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente avaliador de posts",
        instruction="""
            Você é um avaliador de controle de qualidade, você deve priorizar
            as médias de avaliações, você deve usar a feramenta de busca do Google (google_search).
            Guarde os preços e inclua na tabela.
            Escolha até 5 produtos que tenha a melhor média de avaliação e faça a planilha.
            """
    )
    entrada_do_agente_avaliador = f"Tópico: {topico}\nPlano de post: {avaliacao_de_post}"

    rascunho = call_agent(redator, entrada_do_agente_avaliador)
    return rascunho

In [17]:
##########################################
# --- Agente 4: RESULTADO --- #
##########################################
def agente_retorno(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente revisor de post para redes sociais.",
        instruction="""
            Você é um vendedor e dará informação da mais completa sobre o produto,
            você é especializado vendas varejista, com foco no melhor móvel, com melhor custo, benefício e com menor prazo de entrega se tiver.
            Por ter um público, entre 20 e 50 anos, use um tom de escrita adequado, respeitoso e confiante.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho avaliado, se estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            Mostre uma tabelha com somente 5 móveis ou produtos e que tenham preço em ordem crescente, com o nome das empresas e suas avaliações positivas.
            Faça hiperlinks dos links de cada produto da pesquisa inserindo na planilha.
            """
    )
    entrada_do_agente_retorno = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"

    texto_final = call_agent(revisor, entrada_do_agente_retorno)
    return texto_final

In [16]:
data_de_hoje = date.today().strftime("%d/%m/%Y")
loop = "play"
print("📜 Iniciando o Sistema de Pesquisa de Preços de móveis ou produtos com avaliação de qualidade com 4 Agentes IA do Google 📜\n")
while loop != "sair" or topico != "SAIR" or topico != "Sair":
  topico = input("🔍 Por favor, digite o tipo do Móvel ou Produto desejado sobre o qual você quer encontrar ou comprar 🔍: ")
  loop = topico
  if topico == "sair" or topico == "SAIR" or topico == "Sair":
    print("O programa foi encerrado com sucesso.")
    break
  if not topico:
    print("🤔 Você não digitou um tipo de móvel ou produto. Por favor, tente novamente ou digite SAIR para finalizar o programa. 🤔\n")
  else :
    print(f"💲 Pronto, vamos buscar pelo seu móvel ou produto e exibir o orçamento 💲: {topico}" )
    moveis_buscados = agente_buscador(topico, data_de_hoje)
    cotador_de_custos = agente_buscador (topico, moveis_buscados)
    avaliador_de_empresas = agente_buscador (topico, cotador_de_custos)
    resultado_do_post = agente_buscador (topico, avaliador_de_empresas)
    print("\n----------------- RESULTADO DOS AGENTES -----------------\n")
    display(to_markdown(resultado_do_post))
    print("----------------------------------------------------------------")

📜 Iniciando o Sistema de Pesquisa de Preços de móveis ou produtos com avaliação de qualidade com 4 Agentes IA do Google 📜

💲 Pronto, vamos buscar pelo seu móvel ou produto e exibir o orçamento 💲: sofá

----------------- RESULTADO DOS AGENTES -----------------



> Muito obrigado pelas informações adicionais! Para garantir que a tabela esteja o mais precisa e útil possível, vou realizar mais algumas buscas no Google com foco em encontrar opções ainda mais baratas, com garantias mais longas ou com melhor reputação, sempre considerando a data de hoje.
> 
> 
> Com base nas minhas buscas atualizadas, aqui está uma tabela revisada com algumas opções de sofás, considerando preço, garantia e reputação, com informações de maio de 2025:
> 
> **Tabela de Sofás Atualizada (Maio de 2025):**
> 
> | Modelo de Sofá                                                                    | Tipo                   | Características                                                                                                                                   | Marca                      | Preço (Estimado) | Onde Comprar                                                                                                                    | Garantia                                                                               | Reputação                                                                                                                                                 |
> | :---------------------------------------------------------------------------------- | :--------------------- | :---------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------- | :--------------- | :------------------------------------------------------------------------------------------------------------------------------ | :-------------------------------------------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------- |
> | Sofá Retrátil e Reclinável Compacto                                                    | Retrátil/Reclinável    | Design compacto e funcional, ideal para espaços menores                                                                                         | Cama InBox                 | R$ 1.659,00      | Cama inBox                                                                                                               | 12 meses                                                                    | Cama inBox é uma marca com boa reputação no mercado de sofás.                                                                                         |
> | Sofá Retrátil 3 Lugares Costa Rica                                                    | Retrátil               | Design moderno em cinza suede, prático e confortável                                                                                           | Selmer Estofados           | R$ 1.100,00      | SofasExpres.com                                                                                                            | Não especificada                                                                        | Selmer Estofados oferece um bom custo-benefício em seus produtos.                                                                                  |
> | Sofá Retrátil e Reclinável 2 Lugares Livia                                            | Retrátil/Reclinável    | Opção barata e funcional, ideal para ambientes menores                                                                                          | Adonai Estofados           | R$ 800,00        | SofasExpres.com                                                                                                            | Não especificada                                                                        | Adonai Estofados oferece opções com bom custo-benefício.                                                                                         |
> | Sofá 4 Lugares Retrátil e Reclinável Confort                                          | Retrátil/Reclinável    | Espaçoso e confortável, ideal para famílias                                                                                                | Confort-Legacy Sofas       | Abaixo de R$1.800 | SofasExpres.com                                                                                                            | Não especificada                                                                        | Confort-Legacy Sofas oferece sofás grandes e funcionais.                                                                                          |
> | Sofá-Cama Casal 3 Lugares Premium Suede                                              | Sofá-Cama              | Prático e bonito, ideal para otimizar espaços                                                                                              | Mobly                      | R$ 1.102,95      | Mobly                                                                                                                  | Não especificada                                                                        | A Mobly é uma loja com boa reputação e variedade de produtos.                                                                                      |
> | Sofá 3 Lugares Cama inBox Belém Retrátil/Reclinável                                 | Retrátil/Reclinável    | Revestido em Suede Velusoft                                                                                                                    | Cama inBox                 | R$ 1.399,00      | Pontofrio                                                                                                              | 12 meses                                                                    | Cama inBox é uma marca com boa reputação no mercado de sofás.                                                                                         |
> | Sofá 3 Lugares DDS Next Pop Retrátil/Reclinável                                     | Retrátil/Reclinável    | Com porta-copos                                                                                                                               | DDS                        | R$ 989,10        | Pontofrio                                                                                                              | Não especificada                                                                        | Informação não verificada                                                                                                                                   |
> | Sofá 3 lugares RUBEN cinzento claro                                                      | Fixo                   | Estrutura de madeira de pinho e aglomerado, assentos de espuma de poliuretano, encostos e braços preenchidos com fibra oca siliconada e espuma picada. Pés metálicos | Atrapamuebles              | R$ 1.800,00      | Atrapamuebles                                                                                                              | 3 anos                                                                                | Materiais de primeira qualidade.                                                                                                                   |
> | Sofá Retrátil Reclinável 3 Lugares Suede Veludo - Arezzo BestHouse                       | Retrátil/Reclinável    | Revestimento de suede veludo                                                                                                                 | BestHouse                   | R$ 1.200,00      | Magazine Luiza                                                                                                         | 3 meses                                                                              | Tecido fácil de limpar.                                                                                                                             |
> | Sofá 4 Lugares Jobim Retrátil e Reclinável Bege 2,30m                                | Retrátil/Reclinável    | Conforto garantido com assento reclinável, fácil manutenção com limpeza simples, design moderno que combina com qualquer decoração                      | (Não especificado)         | (Não especificado) | (Não especificado)                                                                                                            | (Não especificada)                                                                      | Preço acessível, ideal para quem busca economia sem abrir mão da qualidade.                                                                            |
> | Sofá com Porta Copos Retrátil e Reclinável França                                   | Retrátil/Reclinável    | Design moderno, cor cinza elegante, funcionalidades extras como porta-copos retráteis e encosto reclinável, estrutura em madeira de eucalipto e tecido veludo macio | Luizzi França             | (Não especificado) | (Não especificado)                                                                                                            | (Não especificada)                                                                      | Sofisticação e praticidade, material de qualidade.                                                                                                    |
> | Sofá com USB Retrátil e Reclinável Samurai Areia 2,00m                               | Retrátil/Reclinável    | (Não especificado)                                                                                                                            | Netsofas                  | (Não especificado) | (Não especificado)                                                                                                            | (Não especificada)                                                                      | Tecnologia e conforto.                                                                                                                              |
> 
> **Observações Adicionais:**
> 
> *   **Marcas e Lojas:** Algumas marcas e lojas mencionadas com boa reputação incluem Cama InBox, Mobly, e lojas como Magazine Luiza e Pontofrio.
> *   **Reclame Aqui:** Sempre verifique a reputação da loja e da marca no Reclame Aqui antes de tomar uma decisão final.
> *   **Sofás Baratos:** É possível encontrar sofás retráteis baratos na Shopee.
> *  **Sofás para espaços pequenos:** Os modelos retráteis, de canto e com baú são as melhores opções para aproveitar o espaço sem perder o conforto.
> 
> **Recomendações:**
> 
> *   **Visite lojas físicas:** Se possível, visite lojas físicas para testar o conforto dos sofás e verificar a qualidade dos materiais.
> *   **Considere o tamanho:** Meça o espaço disponível na sua sala para garantir que o sofá escolhido se encaixe perfeitamente.
> *   **Leia avaliações:** Leia avaliações de outros clientes sobre os sofás que você está considerando para ter uma ideia da qualidade, conforto e durabilidade.
> *   **Verifique o tipo de tecido:** Escolha um tecido resistente e fácil de limpar, que seja adequado para o seu estilo de vida.
> *   **Atente-se à densidade da espuma:** A densidade da espuma influencia no conforto e na durabilidade do sofá. Espumas com densidade D23 ou superior são geralmente recomendadas.
> 


----------------------------------------------------------------
O programa foi encerrado com sucesso.
